# Day6 RAG 作業（Hybrid Search + Rerank + DeepEval）

> 這份 notebook 會一步一步帶你完成：資料整理 → 建立 Qdrant Hybrid Index → Query Rewrite → Hybrid Retrieve → Qwen3 Rerank → LLM 生成答案 → DeepEval 五指標評估 → 產出 `day6_HW_questions.csv`。

**你的環境資訊（已確認）**
- Qdrant: `http://localhost:6333`
- Embedding API: `https://ws-04.wade0426.me/embed`（回傳 4096 維，欄位 `embeddings`）
- LLM API: `https://ws-03.wade0426.me/v1/chat/completions`，model=`/models/gpt-oss-120b`
- Reranker: 本機 `Qwen3-Reranker-0.6B`（預設路徑 `/home/randy/Qwen3-Reranker-0.6B`）


## 0. 安裝依賴套件

> 第一次跑才需要。若你環境已經裝好，可以跳過。

In [1]:
%pip -q install qdrant-client fastembed transformers torch accelerate sentencepiece deepeval pandas openpyxl python-docx tqdm requests


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. 參數設定（API、路徑、collection 名稱）

> 這一格是整份作業最重要的設定區。之後你只要改這裡，就能換模型或換資料。

In [27]:
import os

# ========== API ==========
QDRANT_URL = "http://localhost:6333"

EMBED_API_URL = "https://ws-04.wade0426.me/embed"
EMBED_TASK_DESCRIPTION = "檢索技術文件"
EMBED_DIM = 4096

LLM_API_URL = "https://ws-06.huannago.com/v1/chat/completions"
LLM_MODEL = "gemma-3-27b-it"

# 如果你之後發現 API 需要 token，就把環境變數 DAY6_API_KEY 設起來
API_KEY = os.getenv("DAY6_API_KEY", "")  # 沒有就留空

# ========== Local Reranker ==========
RERANKER_MODEL_PATH = "/home/randy/Qwen3-Reranker-0.6B"  # 若載入失敗，可改成 /home/randy/Qwen3-Reranker-0.6B/Qwen3-Reranker-0.6B

# ========== Files ==========
QA_DOCX_PATH = "qa_data.docx"                 # 你下載的 qa_data（這裡用你上傳的 docx 版本）
QUESTIONS_XLSX_PATH = "day6_HW_questions.csv.xlsx"     # 題目表（你提供的是 xlsx）
GT_XLSX_PATH = "questions_answer.csv.xlsx"             # 參考答案（ground truth）

# ========== Qdrant ==========
COLLECTION_NAME = "day6_hybrid_search_demo"


## 2. 讀取資料（題目表、參考答案、知識庫）

> `day6_HW_questions` 是你要填回去的 CSV。
> `questions_answer` 是用來做 DeepEval 時的對照答案。
> `qa_data` 是你的知識庫（RAG 的 context 來源）。

In [2]:
import pandas as pd
from docx import Document

df_questions = pd.read_excel(QUESTIONS_XLSX_PATH)
df_gt = pd.read_excel(GT_XLSX_PATH)

doc = Document(QA_DOCX_PATH)
kb_lines = [p.text.strip() for p in doc.paragraphs if p.text.strip()]

print("Questions rows:", len(df_questions))
print("GT rows:", len(df_gt))
print("KB lines:", len(kb_lines))
display(df_questions.head(3))
display(df_gt.head(3))
print("KB preview:", kb_lines[:8])


Questions rows: 30
GT rows: 30
KB lines: 1348


,q_id,questions,answer,Faithfulness,Answer_Relevancy,Contextual_Recall,Contextual_Precision,Contextual_Relevancy
0,1,我不想要紙本帳單了，想改用手機收簡訊看水費，要去哪裡登記？,NaN,NaN,NaN,NaN,NaN,NaN
1,2,家裡的水龍頭打開有一股像游泳池的味道，這是不是水質有問題？,NaN,NaN,NaN,NaN,NaN,NaN
2,3,浴室地板看到紅色的線蟲在動，是自來水管裡面爬出來的嗎？會不會生病？,NaN,NaN,NaN,NaN,NaN,NaN


,q_id,questions,answer
0,1,我不想要紙本帳單了，想改用手機收簡訊看水費，要去哪裡登記？,可以至台水公司「網路e櫃台」或「E帳單申辦」網頁申請簡訊帳單。需經過手機OTP認證，申請成功...
1,2,家裡的水龍頭打開有一股像游泳池的味道，這是不是水質有問題？,這是正常現象。自來水在淨水處理過程中會加氯消毒，以符合飲用水水質標準（0.2~1.0毫克/公...
2,3,浴室地板看到紅色的線蟲在動，是自來水管裡面爬出來的嗎？會不會生病？,紅蟲通常是搖蚊科昆蟲的幼蟲，不太可能是從自來水管線侵入（因為有濾網且有餘氯消毒）。紅蟲多半是...


KB preview: ['什麼是「簡訊帳單」？', '**發布日期**: 2022/09/15', '本公司為響應節能減碳，提供即時帳單通知服務，將現行紙本帳單改以「簡訊」方式發送，點開簡訊所附的帳單連結，並輸入手機號碼及認證碼，完成驗證程序後即可開啟當期帳單。', '來源：https://www.water.gov.tw/ch/Subject/Detail/14398?nodeId=4889', '何時開始實施簡訊帳單服務？', '**發布日期**: 2022/09/15', '自110年1月起實施，首波開放申請對象為「代繳用戶」，於111年9月起簡訊帳單再升級，開放代收用戶(非代繳用戶)也能申請簡訊帳單。', '註：代繳是指「以存款帳戶或信用卡自動扣繳水費」。']


## 3. 把知識庫整理成「可檢索 chunks」

> 你的資料是 FAQ 格式：**問題** → 日期 → 內容 → 來源。
> 這裡會把它切成一筆筆 chunk（每筆含：question / answer / source / full_text）。

小提醒：這是工程作業，不用追求最完美切法；**切得穩、好 debug、能提升召回** 才是王道。

In [3]:
import re
from typing import List, Dict

def is_question_line(s: str) -> bool:
    # 以問號結尾的行，多半是問題
    if s.startswith("來源：") or s.startswith("**發布日期**"):
        return False
    return s.endswith("？") or s.endswith("?")

def parse_faq_blocks(lines: List[str]) -> List[Dict]:
    blocks = []
    cur_q = None
    cur_lines = []
    for line in lines:
        if is_question_line(line):
            # flush previous
            if cur_q is not None and cur_lines:
                blocks.append({"question": cur_q, "body_lines": cur_lines})
            cur_q = line
            cur_lines = []
        else:
            if cur_q is not None:
                cur_lines.append(line)

    if cur_q is not None and cur_lines:
        blocks.append({"question": cur_q, "body_lines": cur_lines})

    # post-process: extract source(s), remove noisy markers
    out = []
    for i, b in enumerate(blocks):
        body = b["body_lines"]
        sources = [x.replace("來源：", "").strip() for x in body if x.startswith("來源：")]
        body_clean = [x for x in body if not x.startswith("來源：") and not x.startswith("**發布日期**")]
        answer = "\n".join(body_clean).strip()
        full_text = f"Q: {b['question']}\nA: {answer}"
        if sources:
            full_text += "\nSOURCE: " + " | ".join(sources)
        out.append({
            "doc_id": i,
            "question": b["question"],
            "answer": answer,
            "sources": sources,
            "text": full_text
        })
    return out

kb_docs = parse_faq_blocks(kb_lines)
print("Parsed FAQ blocks:", len(kb_docs))
print(kb_docs[0]["text"][:300])


Parsed FAQ blocks: 215
Q: 什麼是「簡訊帳單」？
A: 本公司為響應節能減碳，提供即時帳單通知服務，將現行紙本帳單改以「簡訊」方式發送，點開簡訊所附的帳單連結，並輸入手機號碼及認證碼，完成驗證程序後即可開啟當期帳單。
SOURCE: https://www.water.gov.tw/ch/Subject/Detail/14398?nodeId=4889


## 4. 呼叫 Embedding API（4096 維）

> 這邊會把每個 chunk 的 `text` 轉成向量。

⚠️ 如果你之後遇到 `401/403`，才需要設 `DAY6_API_KEY`，程式會自動帶 `Authorization: Bearer ...`。

In [4]:
import requests
from tqdm import tqdm

def embed_texts(texts, batch_size=16):
    headers = {}
    if API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"

    all_vecs = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        payload = {
            "texts": batch,
            "task_description": EMBED_TASK_DESCRIPTION,
            "normalize": True
        }
        r = requests.post(EMBED_API_URL, json=payload, headers=headers, timeout=120)
        if r.status_code != 200:
            print("Embed status:", r.status_code)
            print("Embed body:", r.text[:500])
        r.raise_for_status()
        vecs = r.json()["embeddings"]
        all_vecs.extend(vecs)

    assert len(all_vecs) == len(texts)
    assert len(all_vecs[0]) == EMBED_DIM, f"Expected {EMBED_DIM}, got {len(all_vecs[0])}"
    return all_vecs

kb_texts = [d["text"] for d in kb_docs]
# 先小量測試，確認 API 正常
_test_vec = embed_texts(kb_texts[:4], batch_size=4)
print("Test OK. dim =", len(_test_vec[0]))


100%|██████████| 1/1 [00:00<00:00,  1.49it/s]

Test OK. dim = 4096


## 5. 建立 Qdrant Hybrid Index（Dense + Sparse BM25）

> **Dense**：用 embedding 做語意檢索。
> **Sparse (BM25)**：用關鍵字檢索（對專有名詞、數字、制度名稱很有用）。
> **Hybrid**：用 RRF 把兩邊的結果融合，通常比單用 dense 穩。

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.http import models as qm

client = QdrantClient(url=QDRANT_URL)

# Dense 向量欄位名稱
DENSE_NAME = "dense"
SPARSE_NAME = "bm25"

def recreate_collection():
    # 刪掉重建（交作業時很方便：重跑不會堆垃圾）
    if COLLECTION_NAME in [c.name for c in client.get_collections().collections]:
        client.delete_collection(COLLECTION_NAME)

    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config={
            DENSE_NAME: qm.VectorParams(size=EMBED_DIM, distance=qm.Distance.COSINE)
        },
        sparse_vectors_config={
            SPARSE_NAME: qm.SparseVectorParams(
                index=qm.SparseIndexParams(on_disk=False),
                modifier=qm.Modifier.IDF   # BM25/IDF 類似配置（配合 Qdrant 的 bm25 sparse encoder）
            )
        }
    )

recreate_collection()
print("Collection ready:", COLLECTION_NAME)


/home/randy/miniforge3/envs/vllm-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Collection ready: day6_hybrid_search_demo


## 6. 產生 Sparse 向量（BM25）並 upsert 到 Qdrant

> 這一步會把 `kb_docs` 寫進 Qdrant：
> - payload：原文、來源、question、answer
> - vectors：dense embedding + sparse BM25

Qdrant 的 BM25 sparse encoder 這裡用 `fastembed`。

In [6]:
from fastembed import SparseTextEmbedding

sparse_model = SparseTextEmbedding(model_name="Qdrant/bm25")

def build_sparse_vectors(texts):
    # fastembed 回傳 iterable，每筆有 .indices / .values
    sv = []
    for emb in sparse_model.embed(texts):
        sv.append(qm.SparseVector(indices=emb.indices, values=emb.values))
    return sv

# 全量 embedding（這一段第一次跑會花時間）
kb_dense = embed_texts(kb_texts, batch_size=16)
kb_sparse = build_sparse_vectors(kb_texts)

points = []
for d, dense_vec, sparse_vec in zip(kb_docs, kb_dense, kb_sparse):
    payload = {
        "doc_id": d["doc_id"],
        "question": d["question"],
        "answer": d["answer"],
        "sources": d["sources"],
        "text": d["text"],
    }
    points.append(
        qm.PointStruct(
            id=d["doc_id"],
            payload=payload,
            vector={
                DENSE_NAME: dense_vec,
                SPARSE_NAME: sparse_vec
            }
        )
    )

client.upsert(collection_name=COLLECTION_NAME, points=points)
print("Upserted points:", len(points))


100%|██████████| 14/14 [01:17<00:00,  5.53s/it]


Upserted points: 215


## 7. Query Rewrite（先把問題改寫成更好檢索的 query）

> Query rewrite 的目的：
> - 把口語問法 → 變成更像「搜尋引擎 query」
> - 把主體/關鍵字補齊（例如：把「簡訊帳單」補出來）

這裡用同一個 LLM endpoint 來做 rewrite。

In [23]:
import requests

def llm_chat(messages, temperature=0.2, max_tokens=512):
    headers = {"Content-Type": "application/json"}
    if API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"

    payload = {
        "model": LLM_MODEL,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
    }

    r = requests.post(LLM_API_URL, headers=headers, json=payload, timeout=120)
    if r.status_code != 200:
        print("LLM status:", r.status_code)
        print("LLM body:", r.text[:1200])
        r.raise_for_status()

    data = r.json()

    # ✅ 兼容各種 chat completion 服務的取法
    content = None

    # A) OpenAI chat.completions 常見
    if isinstance(data, dict) and "choices" in data and data["choices"]:
        ch0 = data["choices"][0]

        # A1) choices[0].message.content
        if isinstance(ch0, dict):
            msg = ch0.get("message")
            if isinstance(msg, dict):
                content = msg.get("content")

            # A2) choices[0].text（有些服務用 text）
            if not content:
                content = ch0.get("text")

    # B) 某些服務會用 output_text / content / response 之類
    if not content:
        for k in ["output_text", "content", "response", "answer"]:
            if isinstance(data, dict) and isinstance(data.get(k), str) and data.get(k).strip():
                content = data[k]
                break

    if not content or not isinstance(content, str):
        print("LLM json (unexpected):", data)
        raise ValueError("LLM response format not recognized. Please paste the printed JSON.")

    return content.strip()


## 8. Hybrid Retrieval（Dense + BM25 → RRF 融合）

> 這裡用 Qdrant 的 `FusionQuery(RRF)`：
> - dense 取語意相似
> - sparse 取關鍵字
> - RRF 融合後回傳 top_k contexts

接下來的 Rerank 會在這個候選集合上做精排。

In [24]:
def rewrite_query(q: str) -> str:
    # 最穩止損：沒有 rewrite 就直接用原問題
    q = (q or "").strip()
    return q

def hybrid_retrieve(query: str, top_k: int = 8, prefetch_k: int = 40):
    # Dense query vector
    q_dense = embed_texts([query], batch_size=1)[0]
    # Sparse query vector
    q_sparse = build_sparse_vectors([query])[0]

    res = client.query_points(
        collection_name=COLLECTION_NAME,
        query=qm.FusionQuery(fusion=qm.Fusion.RRF),
        prefetch=[
            qm.Prefetch(
                query=q_dense,
                using=DENSE_NAME,
                limit=prefetch_k,
            ),
            qm.Prefetch(
                query=q_sparse,
                using=SPARSE_NAME,
                limit=prefetch_k,
            ),
        ],
        limit=top_k,
        with_payload=True,
    )
    hits = res.points
    return hits

hits = hybrid_retrieve(rewrite_query(df_questions.loc[0, "questions"]), top_k=5)
[(h.id, h.score, h.payload["question"]) for h in hits]


100%|██████████| 1/1 [00:00<00:00,  3.23it/s]


[(7, 0.5, '可透過電話申請/取消簡訊帳單嗎？'),
 (2, 0.33333334, '如何申請簡訊帳單？'),
 (3, 0.25, '如何取消簡訊帳單？'),
 (10, 0.2, '如果沒有收到簡訊帳單怎麼辦？'),
 (6, 0.16666667, '可以同時申請電子帳單及簡訊帳單嗎？')]

## 9. Rerank（Qwen3-Reranker-0.6B 本地精排）

> Hybrid 先做「召回」：拿到一堆可能相關的候選 chunk。
> Rerank 再做「精排」：把真正最 relevant 的排到最前面。

這裡用你下載好的 `Qwen3-Reranker-0.6B`。

In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

RERANKER_MODEL_PATH = "/home/randy/Qwen3-Reranker-0.6B"  # 依你實際路徑改

tokenizer = AutoTokenizer.from_pretrained(RERANKER_MODEL_PATH, trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(
    RERANKER_MODEL_PATH, trust_remote_code=True
).to(device).eval()

# ✅ 關鍵：補 pad_token（不然 batch>1 會爆）
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id


@torch.inference_mode()
def rerank(query: str, docs: list[dict], top_k: int = 5):
    """
    docs: list of payload dict，至少要有 'text' 或 'question'
    return: list[(doc_dict, score_float)]  score 越大越相關
    """
    pairs = []
    for d in docs:
        text = d.get("text") or d.get("question") or ""
        pairs.append((query, text))

    # batch tokenize
    inputs = tokenizer(
        [p[0] for p in pairs],
        [p[1] for p in pairs],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    ).to(device)

    logits = model(**inputs).logits  # (B, 2)  ← 你 debug 的就是這個
    probs = torch.softmax(logits, dim=1)       # (B, 2)
    scores = probs[:, 1].detach().cpu().tolist()  # 取第 1 類當相關分數（通常是 yes/relevant）

    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]


# --- demo（你原本那段）
demo_hits = hybrid_retrieve(rewrite_query(df_questions.loc[0, "questions"]), top_k=8)
demo_docs = [h.payload for h in demo_hits]  # payload 內需有 text/question

top = rerank(df_questions.loc[0, "questions"], demo_docs, top_k=5)

# 顯示用：不要 round(list)，round(score) 才對
pretty = [(round(score, 4), (doc.get("question") or doc.get("text") or "")[:80]) for doc, score in top]
pretty


device: cuda


Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at /home/randy/Qwen3-Reranker-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1/1 [00:00<00:00,  1.42it/s]


[(0.8944, '可透過電話申請/取消簡訊帳單嗎？'),
 (0.881, '手機沒有網路時，我可以收得到簡訊帳單嗎？'),
 (0.86, '什麼是「簡訊帳單」？'),
 (0.8517, '可以同時申請電子帳單及簡訊帳單嗎？'),
 (0.848, '如何申請簡訊帳單？')]

## 10. 用 LLM 生成答案（RAG Answering）

> 這裡會把 rerank 後的 top contexts 串起來，餵給 LLM。

關鍵點：
- **只允許用 contexts 裡的內容回答**（提高 Faithfulness）
- 找不到就坦白說「資料庫沒有提到」

In [28]:
def build_context_text(reranked_docs, max_chars=4000):
    ctxs = []
    total = 0
    for d, score in reranked_docs:
        chunk = d["text"]
        if total + len(chunk) > max_chars:
            break
        ctxs.append(chunk)
        total += len(chunk)
    return "\n\n---\n\n".join(ctxs)

def answer_with_rag(user_q: str, top_k_ctx: int = 5):
    rq = rewrite_query(user_q)
    hits = hybrid_retrieve(rq, top_k=10, prefetch_k=50)
    docs = [h.payload for h in hits]
    reranked = rerank(user_q, docs, top_k=top_k_ctx)

    context = build_context_text(reranked, max_chars=4500)

    system = """你是客服型問答助理。你只能使用我提供的【參考資料】作答：
- 若參考資料足以回答：用中文給出精準、可執行的回答
- 若參考資料不足：直接說「參考資料沒有提到」，並說你缺什麼資訊
- 不要編造、不要猜測
"""
    user = f"""【參考資料】\n{context}\n\n【問題】\n{user_q}\n\n請回答："""

    ans = llm_chat(
        [{"role":"system","content": system},
         {"role":"user","content": user}],
        temperature=0.2,
        max_tokens=512
    ).strip()

    # 也把 contexts 回傳，後面 DeepEval 會用到
    used_contexts = [d["text"] for d, _ in reranked]
    return rq, ans, used_contexts

rq, ans, ctxs = answer_with_rag(df_questions.loc[0, "questions"])
print("Rewritten:", rq)
print("Answer:", ans)
print("Contexts used:", len(ctxs))


100%|██████████| 1/1 [00:00<00:00,  2.83it/s]


Rewritten: 我不想要紙本帳單了，想改用手機收簡訊看水費，要去哪裡登記？
Answer: 可至本公司網路e櫃台/E帳單申辦/簡訊帳單/申請簡訊帳單，經手機OTP認證後即申請成功，自當期或下期採簡訊帳單，不再寄送紙本帳單，並享每期水費減免5元(優惠期限至114年12月止)。
Contexts used: 5


## 11. 跑完整 30 題，填回 `day6_HW_questions`

> 這一步會把每題的 answer 產出，並先把 contexts 暫存起來（DeepEval 要用）。

⚠️ 如果你怕 LLM 很慢：
- 先把 `N=3` 跑三題測 pipeline
- 沒問題再改回 30

In [30]:
from tqdm import tqdm

N = min(5, len(df_questions))  # 先跑全部；若要測試可改成 3

answers = []
contexts_map = {}  # q_id -> list[str]

for i in tqdm(range(N)):
    q_id = int(df_questions.loc[i, "q_id"])
    q = str(df_questions.loc[i, "questions"])
    rq, ans, ctxs = answer_with_rag(q, top_k_ctx=5)
    answers.append(ans)
    contexts_map[q_id] = ctxs

df_out = df_questions.copy()
df_out.loc[:N-1, "answer"] = answers
display(df_out.head(5))


100%|██████████| 5/5 [02:57<00:00, 35.56s/it]
/tmp/ipykernel_171533/890858751.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['可至本公司網路e櫃台/E帳單申辦/簡訊帳單/申請簡訊帳單，經手機OTP認證後即申請成功，自當期或下期採簡訊帳單，不再寄送紙本帳單，並享每期水費減免5元(優惠期限至114年12月止)。', '參考資料沒有提到「像游泳池的味道」這種情況。\n\n我缺資訊，無法判斷家裡的水龍頭打開有一股像游泳池的味道是否為水質問題。您可以撥打台水公司客服專線(直撥1910)洽詢。', '1. 所述紅蟲可能是一種搖蚊科昆蟲的幼蟲。\n2. 自來水供水過程中爬蟲等小動物是無法侵入供水管線內，且因自來水處理流程皆加氯消毒，供水管線保有自由餘氯會維持自來水送到用戶家中仍保有消毒能力，抑制微生物生長。\n3. 紅蟲可能是經由水溝排水管爬上來，或蓄水塔不乾淨而滋生，建議檢查排水渠避免積水，減少蚊蟲滋生源頭，並定期清洗蓄水塔，蓄水塔應加蓋上鎖。\n\n參考資料沒有提到喝到這種蟲會不會影響健康。', '1. 逾水費通知單繳費期限即當月22日至次月21日可至超商等代收機構繳費。\n2. 超過7月22日仍未繳費，為加強便民服務，本公司會將6月及8月份二期水費合併，於7月底郵寄二期催繳通知單，用戶可持單於一定繳費期限內至超商等代收機構臨櫃繳費。', '這是因為輸送自來水管線為維持穩定壓力及避免破管，需在局部設置排（吸）氣閥，作用為於水量變化太大使壓力與流量不穩時，排出或吸入空氣調解管線內壓力。因此而使得從水龍頭流出的自來水夾帶大量的氣泡，看起來白濁，請用戶靜置數秒後，氣泡會自然消失變成清澈，這與水質沒有任何關係，請民眾安心使用。']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_out.loc[:N-1, "answer"] = answers

,q_id,questions,answer,Faithfulness,Answer_Relevancy,Contextual_Recall,Contextual_Precision,Contextual_Relevancy
0,1,我不想要紙本帳單了，想改用手機收簡訊看水費，要去哪裡登記？,可至本公司網路e櫃台/E帳單申辦/簡訊帳單/申請簡訊帳單，經手機OTP認證後即申請成功，自當...,NaN,NaN,NaN,NaN,NaN
1,2,家裡的水龍頭打開有一股像游泳池的味道，這是不是水質有問題？,參考資料沒有提到「像游泳池的味道」這種情況。\n\n我缺資訊，無法判斷家裡的水龍頭打開有一股...,NaN,NaN,NaN,NaN,NaN
2,3,浴室地板看到紅色的線蟲在動，是自來水管裡面爬出來的嗎？會不會生病？,1. 所述紅蟲可能是一種搖蚊科昆蟲的幼蟲。\n2. 自來水供水過程中爬蟲等小動物是無法侵入供...,NaN,NaN,NaN,NaN,NaN
3,4,水費單已經過期幾天了，還可以直接去7-11繳錢嗎？還是要去水公司？,1. 逾水費通知單繳費期限即當月22日至次月21日可至超商等代收機構繳費。\n2. 超過7月...,NaN,NaN,NaN,NaN,NaN
4,5,水龍頭流出來的水白白的像牛奶一樣，放一下又變透明，是不是有加什麼化學藥劑？,這是因為輸送自來水管線為維持穩定壓力及避免破管，需在局部設置排（吸）氣閥，作用為於水量變化太...,NaN,NaN,NaN,NaN,NaN


## 12. DeepEval：五個指標評估

你作業要求的指標：
- Faithfulness（忠實度）
- Answer Relevancy（答案相關性）
- Contextual Recall（上下文召回率）
- Contextual Precision（上下文精確度）
- Contextual Relevancy（上下文相關性）

> DeepEval 需要一個「裁判 LLM」。這裡我們直接用同一個 endpoint（`gpt-oss-120b`）當 judge，最省事。

⚠️ `Contextual*` 類指標通常需要 `expected_output`（參考答案）。我們用 `questions_answer` 當 ground truth。

In [35]:
import json
import time
import math
import pandas as pd

from deepeval.test_case import LLMTestCase
from deepeval.metrics import (
    FaithfulnessMetric,
    AnswerRelevancyMetric,
    ContextualRecallMetric,
    ContextualPrecisionMetric,
    ContextualRelevancyMetric,
)
from deepeval.models.base_model import DeepEvalBaseLLM

# -------- 你要確認這個是「chat completions」完整路徑 --------
# 例：LLM_API_URL = "https://ws-06.huannago.com/v1/chat/completions"
# 例：LLM_MODEL   = "gemma-3-27b-it"
assert "chat/completions" in LLM_API_URL, f"LLM_API_URL 看起來不是 chat completions：{LLM_API_URL}"

def _pick_content_from_chat_completion(data: dict):
    """
    兼容多家 OpenAI-compatible 的回傳：
    - choices[0].message.content 正常
    - content=None 的情況：嘗試 reasoning_content / reasoning / text
    """
    try:
        ch0 = data["choices"][0]
    except Exception:
        return None

    # OpenAI / 多數 vLLM 相容格式
    msg = ch0.get("message") or {}
    content = msg.get("content", None)

    if isinstance(content, str) and content.strip():
        return content

    # 有些服務把內容放在其他欄位（很常見的「content=None」）
    for k in ["reasoning_content", "reasoning", "text", "output_text"]:
        v = msg.get(k, None)
        if isinstance(v, str) and v.strip():
            return v

    # 少數服務：choices[0].text
    t = ch0.get("text", None)
    if isinstance(t, str) and t.strip():
        return t

    return None

def llm_chat_raw(messages, temperature=0.0, max_tokens=800, timeout=120):
    """
    你原本 llm_chat 的「原始版」：回傳 dict（不要先 .strip()）
    """
    import requests
    headers = {"Content-Type": "application/json"}
    if "API_KEY" in globals() and API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"

    payload = {
        "model": LLM_MODEL,
        "messages": messages,
        "temperature": float(temperature),
        "max_tokens": int(max_tokens),
    }

    r = requests.post(LLM_API_URL, headers=headers, data=json.dumps(payload), timeout=timeout)
    if r.status_code != 200:
        # 直接把前 300 字印出來，避免 rich 印爆
        raise RuntimeError(f"LLM HTTP {r.status_code}: {r.text[:300]}")
    return r.json()

def llm_chat_json_only(prompt: str, temperature=0.0, max_tokens=900, retries=3, sleep_sec=0.8):
    """
    DeepEval 會自己給 prompt（要求 truths/claims/verdicts...）。
    我們做的事情只有：
    - 再加一道「只回 JSON」的 system 約束
    - content=None 兼容
    - 若不是合法 JSON：重試
    最後一定回傳「字串」（DeepEval 需要）
    """
    sys = (
        "You are a strict JSON generator.\n"
        "Return ONLY one valid JSON object. No markdown. No explanation.\n"
        "Do not wrap in ```.\n"
    )

    last_raw = None
    last_text = None
    for attempt in range(1, retries + 1):
        data = llm_chat_raw(
            [{"role": "system", "content": sys},
             {"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        last_raw = data
        text = _pick_content_from_chat_completion(data)
        last_text = text

        # content=None → 直接視為失敗，重試
        if not isinstance(text, str) or not text.strip():
            time.sleep(sleep_sec)
            continue

        # 先嘗試直接 loads
        try:
            json.loads(text)
            return text
        except Exception:
            # 嘗試「抽出第一個 { 到最後一個 }」救援（常見：前面多一行廢話）
            s = text
            i = s.find("{")
            j = s.rfind("}")
            if i != -1 and j != -1 and j > i:
                candidate = s[i:j+1]
                try:
                    json.loads(candidate)
                    return candidate
                except Exception:
                    pass

        time.sleep(sleep_sec)

    # 真的救不了：把錯誤縮短回傳，讓上層可以記錄（不要 print 超長 raw）
    raise ValueError(f"Judge LLM did not return valid JSON after {retries} retries. "
                     f"content_head={str(last_text)[:200]}")

class RemoteJudgeLLM(DeepEvalBaseLLM):
    """
    DeepEval 的 judge model wrapper
    """
    def load_model(self):
        return None

    def generate(self, prompt: str) -> str:
        # judge 要比較穩，max_tokens 給高一點（避免 JSON 被截斷）
        return llm_chat_json_only(prompt, temperature=0.0, max_tokens=1000, retries=3)

    async def a_generate(self, prompt: str) -> str:
        # deepeval 有些版本會走 async；先簡單同步 fallback
        return self.generate(prompt)

    def get_model_name(self) -> str:
        return f"remote-judge:{LLM_MODEL}"

judge_model = RemoteJudgeLLM()

metrics = [
    FaithfulnessMetric(model=judge_model),
    AnswerRelevancyMetric(model=judge_model),
    ContextualRecallMetric(model=judge_model),
    ContextualPrecisionMetric(model=judge_model),
    ContextualRelevancyMetric(model=judge_model),
]

# 只跑 5 題
N = 5

# df_out：你在 cell 11 已經產生（含 q_id / questions / answer）
# contexts_map：你在 cell 11 存的 q_id -> list[str]
assert "answer" in df_out.columns, "df_out 沒有 answer 欄位，先跑 cell 11"
assert isinstance(contexts_map, dict) and len(contexts_map) > 0, "contexts_map 不存在或是空的，先跑 cell 11"

# ground-truth（expected_output）從 questions_answer.csv（你上傳那份）
df_gt = pd.read_excel("/home/randy/Day6_hw/HW/questions_answer.csv.xlsx")
gt_map = {int(r.q_id): str(r.answer) for r in df_gt.itertuples(index=False)}

def safe_float(x):
    try:
        if x is None: return math.nan
        return float(x)
    except Exception:
        return math.nan

rows = []
for i in range(N):
    q_id = int(df_out.loc[i, "q_id"])
    q = str(df_out.loc[i, "questions"])
    ans = str(df_out.loc[i, "answer"])
    ctxs = contexts_map.get(q_id, [])
    expected = gt_map.get(q_id, "")

    tc = LLMTestCase(
        input=q,
        actual_output=ans,
        expected_output=expected,
        retrieval_context=ctxs,
    )

    row = {
        "q_id": q_id,
        "questions": q,
        "answer": ans,
    }

    for m in metrics:
        name = getattr(m, "name", m.__class__.__name__)
        try:
            m.measure(tc)  # 同步版本（避免 event loop / recursion 問題）
            row[name] = safe_float(getattr(m, "score", None))
            # 有些 metric 會有 reason
            row[f"{name}_reason"] = str(getattr(m, "reason", ""))[:300]
        except Exception as e:
            row[name] = math.nan
            row[f"{name}_reason"] = f"[metric_error] {type(e).__name__}: {str(e)[:200]}"

    rows.append(row)

df_scores = pd.DataFrame(rows)
display(df_scores)

# 如果你要把分數回填到 df_out（前 5 題）
for col in ["FaithfulnessMetric", "AnswerRelevancyMetric", "ContextualRecallMetric", "ContextualPrecisionMetric", "ContextualRelevancyMetric"]:
    if col in df_scores.columns:
        df_out.loc[:N-1, col] = df_scores[col].values

display(df_out.head(N))


/home/randy/miniforge3/envs/vllm-env/lib/python3.11/site-packages/rich/live.py:260: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

,q_id,questions,answer,FaithfulnessMetric,FaithfulnessMetric_reason,AnswerRelevancyMetric,AnswerRelevancyMetric_reason,ContextualRecallMetric,ContextualRecallMetric_reason,ContextualPrecisionMetric,ContextualPrecisionMetric_reason,ContextualRelevancyMetric,ContextualRelevancyMetric_reason
0,1,我不想要紙本帳單了，想改用手機收簡訊看水費，要去哪裡登記？,可至本公司網路e櫃台/E帳單申辦/簡訊帳單/申請簡訊帳單，經手機OTP認證後即申請成功，自當...,1.0,The score is 1.00 because there are no contrad...,0.800000,The score is 0.80 because the response include...,1.0,The score is 1.00 because the entire expected ...,0.700000,The score is 0.70 because while the most relev...,0.400000,The score is 0.40 because the context primaril...
1,2,家裡的水龍頭打開有一股像游泳池的味道，這是不是水質有問題？,參考資料沒有提到「像游泳池的味道」這種情況。\n\n我缺資訊，無法判斷家裡的水龍頭打開有一股...,1.0,The score is 1.00 because there are no contrad...,0.666667,The score is 0.67 because while the response a...,1.0,The score is 1.00 because all sentences in the...,0.916667,The score is 0.92 because while the most relev...,0.272727,The score is 0.27 because the context primaril...
2,3,浴室地板看到紅色的線蟲在動，是自來水管裡面爬出來的嗎？會不會生病？,1. 所述紅蟲可能是一種搖蚊科昆蟲的幼蟲。\n2. 自來水供水過程中爬蟲等小動物是無法侵入供...,1.0,The score is 1.00 because there are no contrad...,1.000000,The score is 1.00 because the response directl...,1.0,The score is 1.00 because the entire expected ...,1.000000,The score is 1.00 because the most relevant no...,0.411765,The score is 0.41 because the context primaril...
3,4,水費單已經過期幾天了，還可以直接去7-11繳錢嗎？還是要去水公司？,1. 逾水費通知單繳費期限即當月22日至次月21日可至超商等代收機構繳費。\n2. 超過7月...,0.5,The score is 0.50 because the actual output in...,0.666667,The score is 0.67 because the response include...,1.0,The score is 1.00 because the entire expected ...,0.887500,The score is 0.89 because while the most relev...,0.576923,The score is 0.58 because the retrieval contex...
4,5,水龍頭流出來的水白白的像牛奶一樣，放一下又變透明，是不是有加什麼化學藥劑？,這是因為輸送自來水管線為維持穩定壓力及避免破管，需在局部設置排（吸）氣閥，作用為於水量變化太...,1.0,The score is 1.00 because there are no contrad...,1.000000,The score is 1.00 because the response directl...,1.0,The score is 1.00 because all sentences in the...,0.500000,The score is 0.50 because while the second nod...,0.200000,The score is 0.20 because the retrieval contex...


,q_id,questions,answer,Faithfulness,Answer_Relevancy,Contextual_Recall,Contextual_Precision,Contextual_Relevancy,FaithfulnessMetric,AnswerRelevancyMetric,ContextualRecallMetric,ContextualPrecisionMetric,ContextualRelevancyMetric
0,1,我不想要紙本帳單了，想改用手機收簡訊看水費，要去哪裡登記？,可至本公司網路e櫃台/E帳單申辦/簡訊帳單/申請簡訊帳單，經手機OTP認證後即申請成功，自當...,NaN,NaN,NaN,NaN,NaN,1.0,0.800000,1.0,0.700000,0.400000
1,2,家裡的水龍頭打開有一股像游泳池的味道，這是不是水質有問題？,參考資料沒有提到「像游泳池的味道」這種情況。\n\n我缺資訊，無法判斷家裡的水龍頭打開有一股...,NaN,NaN,NaN,NaN,NaN,1.0,0.666667,1.0,0.916667,0.272727
2,3,浴室地板看到紅色的線蟲在動，是自來水管裡面爬出來的嗎？會不會生病？,1. 所述紅蟲可能是一種搖蚊科昆蟲的幼蟲。\n2. 自來水供水過程中爬蟲等小動物是無法侵入供...,NaN,NaN,NaN,NaN,NaN,1.0,1.000000,1.0,1.000000,0.411765
3,4,水費單已經過期幾天了，還可以直接去7-11繳錢嗎？還是要去水公司？,1. 逾水費通知單繳費期限即當月22日至次月21日可至超商等代收機構繳費。\n2. 超過7月...,NaN,NaN,NaN,NaN,NaN,0.5,0.666667,1.0,0.887500,0.576923
4,5,水龍頭流出來的水白白的像牛奶一樣，放一下又變透明，是不是有加什麼化學藥劑？,這是因為輸送自來水管線為維持穩定壓力及避免破管，需在局部設置排（吸）氣閥，作用為於水量變化太...,NaN,NaN,NaN,NaN,NaN,1.0,1.000000,1.0,0.500000,0.200000


## 13. 對 30 題全量評估，並把分數回填到 `day6_HW_questions`

> 這裡會把指標分數寫入欄位：
- `Faithfulness`
- `Answer_Relevancy`
- `Contextual_Recall`
- `Contextual_Precision`
- `Contextual_Relevancy`

In [36]:
score_rows = []

for i in tqdm(range(N)):
    q_id = int(df_out.loc[i, "q_id"])
    q = str(df_out.loc[i, "questions"])
    a = str(df_out.loc[i, "answer"])
    ctxs = contexts_map[q_id]
    expected = gt_map.get(q_id, "")
    s = evaluate_one(q_id, q, a, ctxs, expected)

    # map names -> your csv column names
    df_out.loc[i, "Faithfulness"] = s.get("FaithfulnessMetric")
    df_out.loc[i, "Answer_Relevancy"] = s.get("AnswerRelevancyMetric")
    df_out.loc[i, "Contextual_Recall"] = s.get("ContextualRecallMetric")
    df_out.loc[i, "Contextual_Precision"] = s.get("ContextualPrecisionMetric")
    df_out.loc[i, "Contextual_Relevancy"] = s.get("ContextualRelevancyMetric")

display(df_out.head(5))
print(df_out[["Faithfulness","Answer_Relevancy","Contextual_Recall","Contextual_Precision","Contextual_Relevancy"]].mean())


  0%|          | 0/5 [00:00<?, ?it/s]

/home/randy/miniforge3/envs/vllm-env/lib/python3.11/site-packages/rich/live.py:260: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

 20%|██        | 1/5 [02:16<09:07, 136.84s/it]

/home/randy/miniforge3/envs/vllm-env/lib/python3.11/site-packages/rich/live.py:260: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

 40%|████      | 2/5 [04:28<06:40, 133.65s/it]

...

 80%|████████  | 4/5 [10:40<02:48, 168.14s/it]

/home/randy/miniforge3/envs/vllm-env/lib/python3.11/site-packages/rich/live.py:260: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

100%|██████████| 5/5 [12:54<00:00, 154.83s/it]


,q_id,questions,answer,Faithfulness,Answer_Relevancy,Contextual_Recall,Contextual_Precision,Contextual_Relevancy,FaithfulnessMetric,AnswerRelevancyMetric,ContextualRecallMetric,ContextualPrecisionMetric,ContextualRelevancyMetric
0,1,我不想要紙本帳單了，想改用手機收簡訊看水費，要去哪裡登記？,可至本公司網路e櫃台/E帳單申辦/簡訊帳單/申請簡訊帳單，經手機OTP認證後即申請成功，自當...,1.0,0.857143,1.0,0.804167,0.400000,1.0,0.800000,1.0,0.700000,0.400000
1,2,家裡的水龍頭打開有一股像游泳池的味道，這是不是水質有問題？,參考資料沒有提到「像游泳池的味道」這種情況。\n\n我缺資訊，無法判斷家裡的水龍頭打開有一股...,1.0,0.666667,1.0,0.750000,0.272727,1.0,0.666667,1.0,0.916667,0.272727
2,3,浴室地板看到紅色的線蟲在動，是自來水管裡面爬出來的嗎？會不會生病？,1. 所述紅蟲可能是一種搖蚊科昆蟲的幼蟲。\n2. 自來水供水過程中爬蟲等小動物是無法侵入供...,1.0,1.000000,1.0,1.000000,0.411765,1.0,1.000000,1.0,1.000000,0.411765
3,4,水費單已經過期幾天了，還可以直接去7-11繳錢嗎？還是要去水公司？,1. 逾水費通知單繳費期限即當月22日至次月21日可至超商等代收機構繳費。\n2. 超過7月...,1.0,0.666667,1.0,0.887500,0.560000,0.5,0.666667,1.0,0.887500,0.576923
4,5,水龍頭流出來的水白白的像牛奶一樣，放一下又變透明，是不是有加什麼化學藥劑？,這是因為輸送自來水管線為維持穩定壓力及避免破管，需在局部設置排（吸）氣閥，作用為於水量變化太...,1.0,1.000000,1.0,0.500000,0.200000,1.0,1.000000,1.0,0.500000,0.200000


Faithfulness            1.000000
Answer_Relevancy        0.838095
Contextual_Recall       1.000000
Contextual_Precision    0.788333
Contextual_Relevancy    0.368898
dtype: float64


## 14. 輸出作業檔案（CSV + 評估結果備份）

> 依照作業要求，你需要把 `day6_HW_questions.csv` 填完整。
> 這裡同時也輸出一份 `deepeval_scores.csv` 方便你在報告裡貼平均分數或做比較。

In [37]:
OUT_CSV = "day6_HW_questions.csv"
OUT_SCORES = "deepeval_scores.csv"

df_out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

score_cols = ["q_id","Faithfulness","Answer_Relevancy","Contextual_Recall","Contextual_Precision","Contextual_Relevancy"]
df_out[score_cols].to_csv(OUT_SCORES, index=False, encoding="utf-8-sig")

print("Saved:", OUT_CSV, OUT_SCORES)


Saved: day6_HW_questions.csv deepeval_scores.csv


## 15. 你要交的「說明 PDF」可以怎麼寫（建議大綱）

> 這格是**交作業加分點**：你把 DeepEval 的數字拿來說你怎麼調參、怎麼改 pipeline。

建議 PDF 結構（你直接複製改一改就能交）：
1. 系統架構圖（Query Rewrite → Hybrid Retrieve → Rerank → LLM Answer）
2. 資料處理（FAQ 解析與 chunk 設計）
3. Hybrid Search 設計（dense + BM25 + RRF）
4. Rerank 設計（Qwen3-Reranker-0.6B，為什麼能提升 precision）
5. DeepEval 指標說明（五指標的意義）
6. 實驗與優化（例如：
   - top_k_ctx 3→5 對 Faithfulness / Contextual Precision 的影響
   - prefetch_k 20→50 對 Contextual Recall 的影響
   - 是否做 rewrite 對 Answer Relevancy 的影響
)
7. 結論：最終設定與平均分數
